In [63]:
from ngsolve import *
from ngsolve.meshes import MakeStructured2DMesh
from netgen.geom2d import SplineGeometry, unit_square
from ngsolve.webgui import Draw
import time as timeit
from netgen.csg import *

*geometry and mesh*

In [64]:
n = 5
maxdofs = 1e5
c_low = 10
reduced = True; epsilon = 1e-8
# mesh = MakeStructured2DMesh(quads=False, nx=n, ny=n)
mesh = Mesh(unit_square.GenerateMesh(maxh=1 / 8))
#Draw(mesh)

**2D Stokes for Testing**

RT0 + Nedelec1_0 for velocity and velocity trace

In [65]:
# finite element spaces
V = MatrixValued(L2(mesh, order=0), mesh.dim, False)
W = HDiv(mesh, order=0, RT=True, dirichlet=".*")
M = HCurl(mesh, order=0, type1=True, dirichlet=".*")
Q = L2(mesh, order=0, lowest_order_wb=not reduced)
fes = V * W * M * Q
(L, u, uhat, p), (G, v, vhat, q) = fes.TnT()
gfu = GridFunction(fes)
Lh, uh, uhath, ph = gfu.components

n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size

In [66]:
# bilinear and linear forms
def tang(v):
        return v - (v*n)*n

In [67]:
# exact solution
u_exact1 = x ** 2 * (x - 1) ** 2 * 2 * y * (1 - y) * (2 * y - 1)
u_exact2 = y ** 2 * (y - 1) ** 2 * 2 * x * (x - 1) * (2 * x - 1)
u_exact = CF((u_exact1, u_exact2))

L_exactXX = (2 * x * (x - 1) ** 2 * 2 * y * (1 - y) * (2 * y - 1)
                        + x ** 2 * 2 * (x - 1) * 2 * y * (1 - y) * (2 * y - 1))
L_exactXY = (x ** 2 * (x - 1) ** 2 * 2 * (1 - y) * (2 * y - 1)
                        - x ** 2 * (x - 1) ** 2 * 2 * y * (2 * y - 1)
                        + 2 * x ** 2 * (x - 1) ** 2 * 2 * y * (1 - y))
L_exactYX = (y ** 2 * (y - 1) ** 2 * 2 * (x - 1) * (2 * x - 1)
                        + y ** 2 * (y - 1) ** 2 * 2 * x * (2 * x - 1)
                        + 2 * y ** 2 * (y - 1) ** 2 * 2 * x * (x - 1))
L_exactYY = (2 * y * (y - 1) ** 2 * 2 * x * (x - 1) * (2 * x - 1)
                        + y ** 2 * 2 * (y - 1) * 2 * x * (x - 1) * (2 * x - 1))
L_exact = CF((L_exactXX, L_exactXY, L_exactYX, L_exactYY), dims=(2, 2))

p_exact = x * (1 - x) * (1 - y) - 1 / 12

Plain solver

In [68]:
a = BilinearForm(fes, symmetric=False, condense=True)
if reduced:
        a += epsilon * p * q * dx
a += (InnerProduct(L, G) + c_low*u*v - p*div(v) - div(u)*q) * dx
a += (-G*n * ((u*n)*n + tang(uhat)) + L*n * ((v*n)*n + tang(vhat))) * dx(element_boundary=True)

f = LinearForm(fes)
f += (-(4 * y * (1 - y) * (2 * y - 1) * ((1 - 2 * x) ** 2 - 2 * x * (1 - x))
                + 12 * x ** 2 * (1 - x) ** 2 * (1 - 2 * y))
        + (1 - 2 * x) * (1 - y)) * v[0] * dx
f += (-(4 * x * (1 - x) * (1 - 2 * x) * ((1 - 2 * y) ** 2 - 2 * y * (1 - y))
                + 12 * y ** 2 * (1 - y) ** 2 * (2 * x - 1))
        - x * (1 - x)) * v[1] * dx
f += c_low * u_exact * v * dx

def SolveBVP_plain(level, drawResult=False):
    t0 = timeit.time()
    fes.Update()
    gfu.Update()
    a.Assemble()
    f.Assemble()

    t1 = timeit.time()

    # dirichlet BC
    # homogeneous Dirichlet assumed
    f.vec.data += a.harmonic_extension_trans * f.vec
    inv = a.mat.Inverse(fes.FreeDofs(True), inverse='umfpack')
    gfu.vec.data += inv * f.vec
    gfu.vec.data += a.harmonic_extension * gfu.vec
    gfu.vec.data += a.inner_solve * f.vec

    
    t2 = timeit.time()

    print("Assemble: %.2e, Solve: %.2e" % (t1 - t0, t2 - t1))
    if drawResult:
        Draw(uh, mesh)

Project into CR space and solve H(div) scheme

In [69]:
from logging import exception
from ngsolve.krylovspace import CGSolver, MinResSolver, GMResSolver
from prol import *
from mymg import *

V_cr = FESpace('nonconforming', mesh, dirichlet='.*')
fes_cr = V_cr * V_cr
(ux_cr, uy_cr), (vx_cr, vy_cr) = fes_cr.TnT()

u_cr = CF((ux_cr, uy_cr))
v_cr = CF((vx_cr, vy_cr))
GradU_cr = CF((grad(ux_cr), grad(uy_cr)))
GradV_cr = CF((grad(vx_cr), grad(vy_cr)))
divU_cr = grad(ux_cr)[0] + grad(uy_cr)[1]
divV_cr = grad(vx_cr)[0] + grad(vy_cr)[1]

# embedding from fes_cr to fes by L2 projection
mixmass_cr = BilinearForm(trialspace=fes_cr, testspace=fes)
mixmass_cr += (u_cr*n) * (v*n) * dx(element_boundary=True)
mixmass_cr += tang(u_cr) * tang(vhat) * dx(element_boundary=True)

fesMass = BilinearForm(fes)
fesMass += (u*n) * (v*n) * dx(element_boundary=True)
fesMass += tang(uhat) * tang(vhat) * dx(element_boundary=True)

# RT projected mass mat of CR: ET @ RT0Mass @ E
a_cr = BilinearForm(fes_cr)
# NOTE: mass matrix projection on RT!!!!(W)
# a_cr += (InnerProduct(GradU_cr, GradV_cr) + c_low * u_cr * v_cr \
#          - p_cr * divV_cr - divU_cr * q_cr) * dx
a_cr += (InnerProduct(GradU_cr, GradV_cr) 
         + c_low * Interpolate(u_cr, W) * Interpolate(v_cr, W)
         + 1/epsilon * divU_cr * divV_cr) * dx

# ======== CR MG components
et = meshTopology(mesh, mesh.dim)
et.Update()
prolVcr = FacetProlongationTrig2(mesh, et)
a_cr.Assemble()
MG_cr = MultiGrid(a_cr.mat, prolVcr, nc=V_cr.ndof,
                    coarsedofs=fes_cr.FreeDofs(), w1=0.8,
                    nsmooth=4, sm="gs", var=True,
                    he=True, dim=mesh.dim, wcycle=False)

def SolveBVP_CR(level, drawResult=False):
    if not reduced:
        exception("CR PRESSURE HARMONIC EXTENSION NOT FINISHED! USE REDUCED FORM!")
    t0 = timeit.time()
    fes.Update(); fes_cr.Update()
    gfu.Update()
    a.Assemble(); a_cr.Assemble()
    f.Assemble()
    mixmass_cr.Assemble(); fesMass.Assemble()

    udofs = BitArray(fes.ndof)
    udofs[:] = 0
    cur_ndof = V.ndof
    udofs[cur_ndof: cur_ndof+W.ndof] = W.FreeDofs(True)
    cur_ndof += W.ndof
    udofs[cur_ndof: cur_ndof+M.ndof] = M.FreeDofs(True)
    if not reduced:
        cur_ndof += M.ndof
        udofs[cur_ndof: cur_ndof+Q.ndof] = Q.FreeDofs(True)
    # global dofs mass mat => fesMass is diagonal
    # when dim = 2 !!!!!
    fesM_inv = fesMass.mat.CreateSmoother(udofs)
    E = fesM_inv @ mixmass_cr.mat # E: fes_cr => fes
    ET = mixmass_cr.mat.T @ fesM_inv

    # TODO: MG solver for cr !!!!
    if level > 0:
        et.Update()
        pp = [fes_cr.FreeDofs()]
        pp.append(V_cr.ndof)
        pdofs = BitArray(fes_cr.ndof)
        pdofs[:] = 0
        inner = prolVcr.GetInnerDofs(level)
        for j in range(mesh.dim):
            pdofs[j * V_cr.ndof:(j + 1) * V_cr.ndof] = inner
        # he_prol
        pp.append(a_cr.mat.Inverse(pdofs, inverse="umfpack"))
        # bk smoother
        # TODO: WHY THIS IS WRONG???!!!
        # bjac = et.CreateSmoother(a, {"blocktype": "vertexpatch"})
        # pp.append(bjac)
        pp.append(VertexPatchBlocks(mesh, fes_cr))
        # # pp.append(fes.FreeDofs())
        MG_cr.Update(a_cr.mat, pp)
    
    # inv_cr = a_cr.mat.Inverse(fes_cr.FreeDofs(), inverse='umfpack')
    inv_cr = MG_cr

    pre = E @ inv_cr @ ET
    t1 = timeit.time()

    # dirichlet BC
    # homogeneous Dirichlet assumed
    f.vec.data += a.harmonic_extension_trans * f.vec
    # gfu.vec.data += E @ inv_cr @ ET * f.vec
    inv_fes = CGSolver(a.mat, pre, printrates=True, tol=1e-8, maxiter=50)

    gfu.vec.data += inv_fes * f.vec
    gfu.vec.data += a.harmonic_extension * gfu.vec
    gfu.vec.data += a.inner_solve * f.vec
        
    
    # gfu_cr.vec.data += a_cr.mat.Inverse(fes_cr.FreeDofs(True), inverse='umfpack') * f_cr.vec
    t2 = timeit.time()

    print("Assemble: %.2e, Solve: %.2e" % (t1 - t0, t2 - t1))
    if drawResult:
        Draw(uh, mesh)
        # Draw(uh_cr, mesh)

Projection into HDG-P0 and solve H(div) scheme

In [53]:
# from logging import exception
# from ngsolve.la import EigenValues_Preconditioner

# V0 = MatrixValued(L2(mesh), mesh.dim, False)
# W0 = VectorL2(mesh, order=1)
# M0 = FacetFESpace(mesh, dirichlet=".*")
# Q0 = L2(mesh, lowest_order_wb=False)
# fes0 = M0 * M0 * Q0 * V0 * W0
# (uhatx0, uhaty0, p0, L0, u0), (vhatx0, vhaty0, q0, G0, v0) = fes0.TnT()
# uhat0 = CF((uhatx0, uhaty0))
# vhat0 = CF((vhatx0, vhaty0))

# et = meshTopology(mesh, mesh.dim)
# et.Update()
# prolM0 = FacetProlongationTrig(mesh, et)  # Facet prol

# alpha = 1
# a0 = BilinearForm(fes0, symmetric=False, condense=True)
# if reduced:
#     a0 += -epsilon * p0 * q0 * dx
# a0 += (InnerProduct(L0, G0) + c_low * u0 * v0) * dx
# # int on element bd must have L2 projection!!!!!
# # pay attention to the scheme!!!!!
# ir = IntegrationRule(SEGM, 1)
# a0 += (-uhat0 * (G0 * n) + (L0 * n) * vhat0
#           - uhat0 * n * q0 - p0 * vhat0 * n
#           + alpha / h * (u0 - uhat0) * (v0 - vhat0)) * dx(element_boundary=True,
#                                                           intrules={SEGM: ir})
# # embedding from fes0 to fes by L2 projection
# mixmass0 = BilinearForm(trialspace=fes0, testspace=fes)
# mixmass0 += (uhat0*n) * (v*n) * dx(element_boundary=True)
# mixmass0 += tang(uhat0) * tang(vhat) * dx(element_boundary=True)

# # fesMass has been established

# if not reduced:
#     mixmass0 += p0 * q * dx
#     fesMass += p * q * dx

# a0.Assemble()
# pre0MG = MultiGrid(a0.mat, prolM0, nc=M0.ndof,
#                     coarsedofs=fes0.FreeDofs(True), w1=0.8,
#                     nsmooth=4, sm="gs", var=True,
#                     he=True, dim=2, wcycle=False)


# def SolveBVP_HDGP0(level, drawResult):
#     if not reduced:
#         exception("HDGP0 PRESSURE HARMONIC EXTENSION NOT FINISHED! USE REDUCED FORM!")
    
#     t0 = timeit.time()
#     fes.Update(); fes0.Update()
#     gfu.Update()
#     a.Assemble(); a0.Assemble()
#     f.Assemble()
#     mixmass0.Assemble(); fesMass.Assemble()

#     udofs = BitArray(fes.ndof)
#     udofs[:] = 0
#     cur_ndof = V.ndof
#     udofs[cur_ndof: cur_ndof+W.ndof] = W.FreeDofs(True)
#     cur_ndof += W.ndof
#     udofs[cur_ndof: cur_ndof+M.ndof] = M.FreeDofs(True)
#     if not reduced:
#         cur_ndof += M.ndof
#         udofs[cur_ndof: cur_ndof+Q.ndof] = Q.FreeDofs(True)
#     # global dofs mass mat => fesMass is diagonal
#     fesM_inv = fesMass.mat.CreateSmoother(udofs)
#     E0 = fesM_inv @ mixmass0.mat # E0: fes0 => fes
#     ET0 = mixmass0.mat.T @ fesM_inv
#     t1 = timeit.time()

#     ## Update HDG-P0 MG
#     if level > 0:
#         et.Update()
#         pp = [fes0.FreeDofs(True)]
#         pp.append(M0.ndof)
#         pdofs = BitArray(fes0.ndof)
#         pdofs[:] = 0
#         inner = prolM0.GetInnerDofs(level)
#         for j in range(mesh.dim):
#             pdofs[j * M0.ndof:(j + 1) * M0.ndof] = inner
#         # he_prol
#         pp.append(a0.mat.Inverse(pdofs, inverse="sparsecholesky"))
#         # bk smoother
#         bjac = et.CreateSmoother(a0, {"blocktype": "vertexpatch"})
#         pp.append(bjac)
#         pre0MG.Update(a0.mat, pp)
#     t2 = timeit.time()
#     # estimate condition number
#     # lams = np.array([1, 1])
#     # inv = a0.mat.Inverse(fes0.FreeDofs(True), inverse='umfpack')
#     lams = EigenValues_Preconditioner(mat=a0.mat, pre=pre0MG)
#     # inv0 = CGSolver(a0.mat, pre0MG, printrates=False, tol=1e-8, maxiter=100)
#     t21 = timeit.time()
#     # dirichlet BC
#     # homogeneous Dirichlet assumed
#     f.vec.data += a.harmonic_extension_trans * f.vec
#     pre = E0 @ pre0MG @ ET0
#     inv = CGSolver(a.mat, pre, printrates=False, tol=1e-8, maxiter=100)
#     gfu.vec.data += inv * f.vec
#     it = inv.iterations
#     # gfu0.vec.data += a0.mat.Inverse(fes0.FreeDofs(True), inverse='sparsecholesky') * f0.vec
#     # it = 1
#     gfu.vec.data += a.harmonic_extension * gfu.vec
#     gfu.vec.data += a.inner_solve * f.vec
#     t3 = timeit.time()

#     print(f"Time to find EIG Val: {t21 - t2:.1E}, MAX PREC LAM: {max(lams):.1E}; MIN PREC LAM: {min(lams):.1E}")
#     print("IT: %2.0i" % (it), "cond: %.2e" % (max(lams) / min(lams)),
#           "Assemble: %.2e Prec: %.2e Solve: %.2e" % (t1 - t0, t2 - t1, t3 - t21))
#     if drawResult:
#         Draw(uh, mesh)


Convergence rate test, 1st order convergence expected for Lh, uh and ph

In [70]:
drawResult = False
SolveBVP = SolveBVP_CR

print(f'===== Reduced: {reduced}, c_low: {c_low} =====')
SolveBVP(0, drawResult)
print(f'level: {0}')
L2_uErr = sqrt(Integrate((uh - u_exact) * (uh - u_exact), mesh))
L2_LErr = sqrt(Integrate(InnerProduct((Lh - L_exact), (Lh - L_exact)), mesh))
L2_divErr = sqrt(Integrate(div(uh) * div(uh), mesh))
print(f"uh L2-error: {L2_uErr:.3E}")
print(f"Lh L2-error: {L2_LErr:.3E}")
print(f'uh divErr: {L2_divErr:.1E}')
print('==============================')
prev_uErr = L2_uErr
prev_LErr = L2_LErr
u_rate, L_rate = 0, 0
level = 1
while True:
    with TaskManager():
        mesh.ngmesh.Refine()
        # exit if total global dofs exceed a tol
        M.Update()
        if (M.ndof * mesh.dim > maxdofs):
            print(M.ndof * mesh.dim)
            break
        print(f'level: {level}')
        SolveBVP(level, drawResult)
        # ===== convergence check =====
        meshRate = 2
        L2_uErr = sqrt(Integrate((uh - u_exact) * (uh - u_exact), mesh))
        L2_LErr = sqrt(Integrate(InnerProduct((Lh - L_exact), (Lh - L_exact)), mesh))
        L2_divErr = sqrt(Integrate(div(uh) * div(uh), mesh))
        u_rate = log(prev_uErr / L2_uErr) / log(meshRate)
        L_rate = log(prev_LErr / L2_LErr) / log(meshRate)
        print(f"uh L2-error: {L2_uErr:.3E}, uh conv rate: {u_rate:.2E}")
        print(f"Lh L2-error: {L2_LErr:.3E}, Lh conv rate: {L_rate:.2E}")
        print(f'uh divErr: {L2_divErr:.1E}')
        print('==============================')
        prev_uErr = L2_uErr
        prev_LErr = L2_LErr
        level += 1

===== Reduced: True, c_low: 10 =====
CG iteration 1, residual = 0.06443038333258734     
CG iteration 2, residual = 1.2724869621404237e-05     
CG iteration 3, residual = 6.23778666127024e-09     
CG iteration 4, residual = 5.552027540258493e-13     
Assemble: 1.12e-02, Solve: 1.30e-03
level: 0
uh L2-error: 2.017E-03
Lh L2-error: 2.280E-02
uh divErr: 6.4E-10
level: 1
CG iteration 1, residual = 0.06592655318750915     
CG iteration 2, residual = 0.004377785392450255     
CG iteration 3, residual = 0.00014597062914171774     
CG iteration 4, residual = 4.4160749991602674e-05     
CG iteration 5, residual = 2.67810669715522e-05     
CG iteration 6, residual = 1.5869398149333327e-06     
CG iteration 7, residual = 3.487881178804309e-07     
CG iteration 8, residual = 3.198629266144919e-07     
CG iteration 9, residual = 1.2972292834834448e-08     
CG iteration 10, residual = 1.1904880683052682e-09     
CG iteration 11, residual = 2.4499294955931663e-08     
CG iteration 12, residual = 1.27